In [1]:
import pdfquery as pq
import pandas as pd
import requests
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean
from numpy import std
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB


Scrape PDFs

In [2]:
def scrapePdf(pdfPath):
    pdf = pq.PDFQuery(pdfPath)
    pdf.load()
    pdfData = pd.DataFrame({
        "countryName": pdf.pq('LTTextLineHorizontal:overlaps_bbox("138.98, 686.86, 211.513, 697.9")').text(),
        "EPRETRSectorCode": pdf.pq('LTTextLineHorizontal:overlaps_bbox("180.26, 643.3, 185.857, 654.34")').text(),
        "eptrSectorName":
            pdf.pq('LTTextLineHorizontal:overlaps_bbox("210.29, 643.3, 471.936, 654.34")').text().split(" ", 1)[1],
        "EPRTRAnnexIMainActivityCode":
            pdf.pq('LTTextLineHorizontal:overlaps_bbox("52.8, 614.26, 157.097, 625.3")').text().split(" ", 1)[1],
        "FacilityInspireID":
            pdf.pq('LTTextLineHorizontal:overlaps_bbox("52.8, 715.9, 266.172, 726.94")').text().split(" ", 1)[1],
        "facilityName":
            pdf.pq('LTTextLineHorizontal:overlaps_bbox("52.8, 730.42, 343.076, 741.46")').text().split(" ", 2)[2],
        "City": pdf.pq('LTTextLineHorizontal:overlaps_bbox("138.98, 672.34, 221.195, 683.38")').text(),
        "CITY ID": pdf.pq('LTTextLineHorizontal:overlaps_bbox("138.98, 175.7, 312.518, 186.74")').text(),
        "targetRelease": pdf.pq('LTTextLineHorizontal:overlaps_bbox("138.98, 570.67, 154.094, 581.71")').text(),
        "pollutant": pdf.pq('LTTextLineHorizontal:overlaps_bbox("306.17, 570.67, 406.325, 581.71")').text(),
        "DAY": pdf.pq('LTTextLineHorizontal:overlaps_bbox("174.62, 527.11, 185.857, 538.15")').text(),
        "MONTH": pdf.pq('LTTextLineHorizontal:overlaps_bbox("347.47, 527.11, 384.086, 538.15")').text().split(" ", 1)[
            0],
        "reportingYear": pdf.pq('LTTextLineHorizontal:overlaps_bbox("461.38, 527.11, 483.897, 538.15")').text(),
        "CONTINENT": pdf.pq('LTTextLineHorizontal:overlaps_bbox("437.02, 686.86, 473.96, 697.9")').text(),
        "max_wind_speed":
            pdf.pq('LTTextLineHorizontal:overlaps_bbox("52.8, 452.11, 185.806, 463.15")').text().split(" ", 1)[1],
        "avg_wind_speed":
            pdf.pq('LTTextLineHorizontal:overlaps_bbox("316.87, 452.11, 483.846, 463.15")').text().split(" ", 2)[2],
        "min_wind_speed":
            pdf.pq('LTTextLineHorizontal:overlaps_bbox("316.87, 452.11, 483.846, 463.15")').text().split(" ", 1)[0],
        "max_temp": pdf.pq('LTTextLineHorizontal:overlaps_bbox("144.02, 388.01, 185.806, 399.05")').text(),
        "avg_temp": pdf.pq('LTTextLineHorizontal:overlaps_bbox("442.06, 388.01, 483.846, 399.05")').text(),
        "min_temp":
            pdf.pq('LTTextLineHorizontal:overlaps_bbox("311.23, 388.01, 405.714, 399.05")').text().split(" ", 1)[0],
        "DAYS WITH FOG": pdf.pq('LTTextLineHorizontal:overlaps_bbox("174.62, 323.93, 185.857, 334.97")').text(),
        "REPORTER NAME":
            pdf.pq('LTTextLineHorizontal:overlaps_bbox("356.95, 233.81, 504.706, 244.85")').text().split(":", 1)[1]
    }, index=[0])
    
    pdfData.rename(columns={'DAYS WITH FOG': 'DAY WITH FOGS'}, inplace=True)
    pdfData.rename(columns={'eptrSectorName': 'eprtrSectorName'}, inplace=True)
    return pdfData


def getPdfNames():
    pdfsPath = './pdfs/'
    files = [f for f in os.listdir(pdfsPath) if os.path.isfile(os.path.join(pdfsPath, f))]
    for file in files:
        if file.split(".")[1] != 'pdf':
            files.remove(file)
    files = map(lambda file: pdfsPath + file, files)
    return files

pdfData = pd.DataFrame()
files = getPdfNames()
for file in files:
    pdfData = pd.concat([pdfData, scrapePdf(file)], axis=0)

Get JSONs

In [6]:
def jsonToDataframe(url):
    print(url)
    resp = requests.get(url=url)
    data = resp.json()
    df = pd.DataFrame()
    for count, row in enumerate(data):
        entry = pd.DataFrame(data[count], index=[count])
        df = pd.concat([df, entry], axis='rows')
    return df


urls = ['http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/first',
        'http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/second',
        'http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/third']
jsonData = pd.DataFrame()
for url in urls:
    temp = jsonToDataframe(url)
    jsonData = pd.concat([jsonData, temp], axis='rows')
#c
jsonData.drop(columns=['EPRTRSectorCode'], inplace=True)
jsonData.drop(columns=[''], inplace=True)
print(jsonData.columns)

http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/first
http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/second
http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/third
Index(['CITY ID', 'CONTINENT', 'City', 'DAY', 'DAY WITH FOGS',
       'EPRTRAnnexIMainActivityCode', 'EPRTRAnnexIMainActivityLabel',
       'FacilityInspireID', 'MONTH', 'REPORTER NAME', 'avg_temp',
       'avg_wind_speed', 'countryName', 'eprtrSectorName', 'facilityName',
       'max_temp', 'max_wind_speed', 'min_temp', 'min_wind_speed', 'pollutant',
       'reportingYear', 'targetRelease'],
      dtype='object')


Get Csvs

In [7]:
def getcode(label, dictionary):
    # based on data from here: https://iir.umweltbundesamt.de/2021/general/point_sources/start
    if label == 'Chemical installations for the production on an industrial scale of basic organic chemicals: Organometallic compounds':
        return '4(a)(vii)'
    return dictionary[label]

df1 = pd.read_csv("./csvs/train1.csv")
df2 = pd.read_csv("./csvs/train2.csv", sep=';')
frames = [df1,df2]
csvData = pd.concat(frames)

names_list = csvData.columns.to_list()

aux = 0
for element in names_list:
    if csvData[element].isnull().values.any():
        aux = aux + 1
        print(csvData[element].isnull().values.any())
if aux > 0:
    print('existing nulls')
else:
    print('no existing nulls')
csvData['EPRTRAnnexIMainActivityCode'] = csvData.apply(lambda row: getcode(row['EPRTRAnnexIMainActivityLabel'], dict(zip(jsonData['EPRTRAnnexIMainActivityLabel'], jsonData['EPRTRAnnexIMainActivityCode']))), axis=1)
csvData.drop(columns=['EPRTRAnnexIMainActivityLabel'], inplace=True)
jsonData.drop(columns=['EPRTRAnnexIMainActivityLabel'], inplace=True)

no existing nulls


Unify data

In [17]:
data = pd.concat([jsonData, csvData, pdfData], axis='rows')


data.drop(columns=['FacilityInspireID'], inplace=True)
data.drop(columns=['facilityName'], inplace=True)
data.drop(columns=['targetRelease'], inplace=True)
data.drop(columns=['CONTINENT'], inplace=True)
data.drop(columns=['REPORTER NAME'], inplace=True)
data.drop(columns=['CITY ID'], inplace=True)
data.drop(columns=['EPRETRSectorCode'], inplace=True)

We swap min and max values for wind and temperature when min values are greater than max values

In [18]:
data.loc[data['min_wind_speed'] > data['max_wind_speed'], ['min_wind_speed','max_wind_speed']] = data.loc[data['min_wind_speed'] > data['max_wind_speed'], ['max_wind_speed','min_wind_speed']].values
data.loc[data['min_temp'] > data['max_temp'], ['min_temp','max_temp']] = data.loc[data['min_temp'] > data['max_temp'], ['max_temp','min_temp']].values


Transformations:

Replace max, min and average temperature and winds for pdf values because they don't have sense

In [19]:
head = data.head(-82)
uk = head.loc[head['countryName'] == 'United Kingdom']
uk_max_wind = uk['max_wind_speed'].astype(float).mean()
uk_min_wind = uk['min_wind_speed'].astype(float).mean()
uk_avg_wind = uk['avg_wind_speed'].astype(float).mean()
uk_max_temp = uk['max_temp'].astype(float).mean()
uk_min_temp = uk['min_temp'].astype(float).mean()
uk_avg_temp = uk['avg_temp'].astype(float).mean()
tail = data.tail(82)
tail = tail.assign(max_wind_speed=uk_max_wind)
tail = tail.assign(min_wind_speed=uk_min_wind)
tail = tail.assign(avg_wind_speed=uk_avg_wind)
tail = tail.assign(max_temp=uk_max_temp)
tail = tail.assign(min_temp=uk_min_temp)
tail = tail.assign(avg_temp=uk_avg_temp)

data = pd.concat([head, tail], axis='rows')

Encoding:

In [20]:
le_general = LabelEncoder()
le_pollutant = LabelEncoder()
data['pollutant'] = le_pollutant.fit_transform(data['pollutant'])
data['countryName'] = le_general.fit_transform(data['countryName'])
data['eprtrSectorName'] = le_general.fit_transform(data['eprtrSectorName'])
data['City'] = le_general.fit_transform(data['City'])
data['EPRTRAnnexIMainActivityCode'] = le_general.fit_transform(data['EPRTRAnnexIMainActivityCode'])
features = data.columns.tolist()
features.remove('pollutant')
X = data[features]
y = data['pollutant']

Creation of test and training dataset

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

KNN Model training:

In [22]:
model = KNeighborsClassifier(n_neighbors=5)

# Train the model using the training sets
model.fit(X_train, y_train)

#Predict Output
y_pred = model.predict(X_test)
conf_mat = metrics.confusion_matrix(y_test, y_pred)
print("Confussion's matrix: \n{}".format(conf_mat))
print("Accuracy score: {}".format(metrics.accuracy_score(y_test, y_pred)))
print("Precission score: {}".format(metrics.precision_score(y_test, y_pred, average='micro')))
print("Recall score: {}".format(metrics.recall_score(y_test, y_pred, average='micro')))
print("F1 score: {}".format(metrics.f1_score(y_test, y_pred, average='micro')))

Confussion's matrix: [[2388  414 1825]
 [ 505 2556  329]
 [2020  485 2620]]
Accuracy score: 0.5755592756049308
Precission score: 0.5755592756049308
Recall score: 0.5755592756049308
F1 score: 0.5755592756049308


Gaussian Naïve Bayes training:

In [23]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
conf_mat = metrics.confusion_matrix(y_test, y_pred)
print("Confussion's matrix: \n{}".format(conf_mat))
print("Accuracy score: {}".format(metrics.accuracy_score(y_test, y_pred)))
print("Precission score: {}".format(metrics.precision_score(y_test, y_pred, average='micro')))
print("Recall score: {}".format(metrics.recall_score(y_test, y_pred, average='micro')))
print("F1 score: {}".format(metrics.f1_score(y_test, y_pred, average='micro')))


Confussion's matrix: 
[[ 656  904 3067]
 [  83 2498  809]
 [ 628 1017 3480]]
Accuracy score: 0.5047937908994065
Precission score: 0.5047937908994065
Recall score: 0.5047937908994065
F1 score: 0.5047937908994065


Random Forest training: 

In [24]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
conf_mat = metrics.confusion_matrix(y_test, y_pred)
conf_mat = metrics.confusion_matrix(y_test, y_pred)
print("Confussion's matrix: \n{}".format(conf_mat))
print("Accuracy score: {}".format(metrics.accuracy_score(y_test, y_pred)))
print("Precission score: {}".format(metrics.precision_score(y_test, y_pred, average='micro')))
print("Recall score: {}".format(metrics.recall_score(y_test, y_pred, average='micro')))
print("F1 score: {}".format(metrics.f1_score(y_test, y_pred, average='micro')))

feature_imp = pd.Series(clf.feature_importances_, index=features).sort_values(ascending=False)
print("Features weight: {}".format(feature_imp.to_string()))

FP_0 = conf_mat[1][0] + conf_mat[2][0]
FN_0 = conf_mat[0][1] + conf_mat[0][2]
FP_1 = conf_mat[0][1] + conf_mat[2][1]
FN_1 = conf_mat[1][0] + conf_mat[1][2]
FP_2 = conf_mat[0][2] + conf_mat[1][2]
FN_2 = conf_mat[2][0] + conf_mat[2][1]

FP = FP_0 + FP_1 + FP_2
FN = FN_0 + FN_1 + FN_2
print("Falses positives: {}".format(FP))
print("Falses negatives: {}".format(FN))

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(clf, data.drop('pollutant', axis=1), data['pollutant'], scoring='f1_micro', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))


Confussion's matrix: 
[[2643   85 1899]
 [ 189 2904  297]
 [1517  115 3493]]
Accuracy score: 0.6878709481053112
Precission score: 0.6878709481053112
Recall score: 0.6878709481053112
F1 score: 0.6878709481053112
Features weight: EPRTRAnnexIMainActivityCode    0.221931
eprtrSectorName                0.107912
min_wind_speed                 0.068798
max_wind_speed                 0.068713
City                           0.068150
avg_wind_speed                 0.068113
min_temp                       0.065433
max_temp                       0.065282
avg_temp                       0.064712
DAY                            0.052260
countryName                    0.044333
reportingYear                  0.040168
MONTH                          0.038129
DAY WITH FOGS                  0.026066
Falses positives: 4102
Falses negatives: 4102
Accuracy: 0.694 (0.005)


Let's talk about metrics: We get a reasonable better F1 score compared to others. Important to mark that different scores are equal because we have the same number of false positives and false negatives. We check the weight from the different features: activity code and sector name are the most valuable. We do a cross validation to obtain a more robust result from our model. 

Predictions output:

In [32]:
# read test data
test = pd.read_csv('test_x.csv')
#test.rename(columns={'EPRTRSectorCode': 'eprtrSectorName'}, inplace=True)
test_index = test['test_index']
for col in test.columns:
    if col not in data.columns:
        test.drop(columns=[col], inplace=True)
        #print('Dropped ' + str(col))
test = test[features]
#test.rename(columns={'eprtrSectorName': 'removed'}, inplace=True)
print(test.head())
test['countryName'] = le_general.fit_transform(test['countryName'])
test['eprtrSectorName'] = le_general.fit_transform(test['eprtrSectorName'])
test['City'] = le_general.fit_transform(test['City'])
test['EPRTRAnnexIMainActivityCode'] = le_general.fit_transform(test['EPRTRAnnexIMainActivityCode'])

predictions = clf.predict(test)
predictions = le_pollutant.inverse_transform(predictions)
final_answer = pd.DataFrame()
final_answer['test_index'] = test_index
final_answer['prediction'] = pd.Series(predictions)

def getPollutantCode(row):
    match row['prediction']:
        case 'Nitrogen oxides (NOX)':
            pollutant = '0'
        case 'Carbon dioxide (CO2)':
            pollutant = '1'
        case 'Methane (CH4)':
            pollutant = '2'
        case _:
            pollutant = '-1'
    return pollutant


final_answer['pollutant'] = final_answer.apply(lambda row: getPollutantCode(row), axis=1)
final_answer.to_csv('predictions.csv', index=False)
final_answer.to_json('predictions.json', orient='records')

        City  DAY  DAY WITH FOGS EPRTRAnnexIMainActivityCode  MONTH  \
0  Rydułtowy   16              1                        3(a)      8   
1   Diekirch   22              0                        5(d)     11   
2  Eemshaven   19              2                        1(c)      9   
3        BRO   17              2                        5(d)      7   
4    SETÚBAL   23              2                        1(c)      6   

    avg_temp  avg_wind_speed  countryName                  eprtrSectorName  \
0  11.381181       14.855940       Poland                 Mineral industry   
1   8.840137       17.623877   Luxembourg  Waste and wastewater management   
2   8.403322       15.541979  Netherlands                    Energy sector   
3   7.571596       17.458113       Sweden  Waste and wastewater management   
4  11.548033       20.532473     Portugal                    Energy sector   

    max_temp  max_wind_speed   min_temp  min_wind_speed  reportingYear  
0  10.278561       14.080054  1